In [4]:
import pandas as pd
import numpy as np
import h5py

import astropy.constants as const
import astropy.units as u
from astropy.coordinates import Galactocentric, ICRS, CartesianRepresentation, GeocentricTrueEcliptic
from astropy.coordinates import SkyCoord

In [5]:
hf_in = h5py.File('Galaxies/Galaxy1/galaxy_binaries.hdf5', 'r')

In [34]:
hf_in['sky']['igb']['cat']['Amplitude'][()]

array([2.01228708e-25, 8.12722687e-26, 1.59415752e-25, ...,
       1.74522518e-25, 1.81236917e-25, 2.13338822e-25], shape=(10000,))

In [32]:
hf_in['sky']['dgb']['cat']['Amplitude'][:]

array([2.67149037e-24, 2.23840646e-24, 2.46040408e-24, ...,
       3.51458264e-24, 2.57028404e-24, 2.05760092e-24], shape=(10000,))

In [35]:
hf_in['sky']['vgb']['cat']['Amplitude'][:]

array([1.57227096e-22, 2.42552879e-23, 2.63856100e-22, 2.82911618e-22,
       4.22389998e-23, 4.52629258e-23, 2.99070069e-22, 9.92075373e-23,
       8.90872482e-23, 1.49514755e-23, 6.37834011e-23, 1.28538211e-22,
       5.48469369e-23, 3.43896264e-22, 4.15074693e-22, 1.32550065e-23,
       5.70003671e-23, 1.21650155e-22, 1.15121212e-22, 4.41469900e-23,
       9.06702280e-23, 2.49034456e-23, 1.06888173e-22, 4.40722619e-22,
       8.42989213e-23, 1.09726229e-22, 1.01259366e-22, 1.72548564e-22,
       1.12976461e-22, 3.20175062e-23, 1.59930233e-22, 6.16870115e-23,
       1.08573720e-22, 2.38395760e-22, 1.61905414e-22, 6.05884776e-23])

In [57]:
dat

,bin_num,mass_1,mass_2,kstar_1,kstar_2,sep,met,tphys,rad_1,rad_2,xGx,yGx,zGx,FIRE_index,f_gw,dist_sun
436,2690549,0.495216,0.400994,10.0,10.0,1.567830,0.000149,10454.115503,0.014301,0.015929,-19.244278,-12.825714,-17.227786,10676022,0.000130,34.184220
1094,1334336,0.455983,0.383181,10.0,10.0,1.129939,0.000149,11622.801374,0.014941,0.016277,-9.398793,9.526395,-4.527165,11950352,0.000497,11.537129
1688,1624731,0.433239,0.315422,10.0,10.0,0.514922,0.000149,13124.013916,0.015335,0.017773,-30.244819,-2.081101,-48.116258,6553631,0.000677,59.712457
2225,4754452,0.450991,0.428632,10.0,10.0,1.421920,0.000162,9420.423401,0.015026,0.015417,135.468392,-7.342160,67.772007,8950098,0.000438,151.453785
2436,4195200,0.460459,0.316052,10.0,10.0,0.576520,0.000175,12836.305432,0.014865,0.017758,-12.065234,10.672532,-1.526126,8882005,0.001296,13.031714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1606756,4894129,1.250921,0.542698,12.0,11.0,2.615819,9.572804,1260.624530,0.004986,0.013582,1.673735,2.706960,-2.133633,11416696,0.000124,6.033043
1606816,553426,1.180926,0.563383,12.0,11.0,2.710438,9.703711,975.096463,0.005923,0.013285,-0.028703,0.212830,-0.034673,1156864,0.000183,7.840230
1606850,7262088,1.379411,0.534644,12.0,11.0,2.488743,9.783309,1428.647945,0.002753,0.013700,7.185729,-1.315174,-0.212167,13087571,0.000125,11.610430
1606919,3491668,1.234470,0.403945,12.0,10.0,2.294323,10.003275,1675.340897,0.005216,0.015873,2.584891,0.997303,0.061882,6035096,0.000150,7.521060


In [65]:
dat.index

Index([    436,    1094,    1688,    2225,    2436,    3327,    3763,    6157,
          6261,    6618,
       ...
       1606675, 1606695, 1606696, 1606723, 1606733, 1606756, 1606816, 1606850,
       1606919, 1607109],
      dtype='int64', length=613089)

In [75]:
dat = pd.read_hdf('Galaxies/LISA_band_FZ_alpha25_Z.hdf', key='data')
dat

,bin_num,mass_1,mass_2,kstar_1,kstar_2,sep,met,tphys,rad_1,rad_2,xGx,yGx,zGx,FIRE_index,f_gw,dist_sun
436,2690549,0.495216,0.400994,10.0,10.0,1.567830,0.000149,10454.115503,0.014301,0.015929,-19.244278,-12.825714,-17.227786,10676022,0.000130,34.184220
1094,1334336,0.455983,0.383181,10.0,10.0,1.129939,0.000149,11622.801374,0.014941,0.016277,-9.398793,9.526395,-4.527165,11950352,0.000497,11.537129
1688,1624731,0.433239,0.315422,10.0,10.0,0.514922,0.000149,13124.013916,0.015335,0.017773,-30.244819,-2.081101,-48.116258,6553631,0.000677,59.712457
2225,4754452,0.450991,0.428632,10.0,10.0,1.421920,0.000162,9420.423401,0.015026,0.015417,135.468392,-7.342160,67.772007,8950098,0.000438,151.453785
2436,4195200,0.460459,0.316052,10.0,10.0,0.576520,0.000175,12836.305432,0.014865,0.017758,-12.065234,10.672532,-1.526126,8882005,0.001296,13.031714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1606756,4894129,1.250921,0.542698,12.0,11.0,2.615819,9.572804,1260.624530,0.004986,0.013582,1.673735,2.706960,-2.133633,11416696,0.000124,6.033043
1606816,553426,1.180926,0.563383,12.0,11.0,2.710438,9.703711,975.096463,0.005923,0.013285,-0.028703,0.212830,-0.034673,1156864,0.000183,7.840230
1606850,7262088,1.379411,0.534644,12.0,11.0,2.488743,9.783309,1428.647945,0.002753,0.013700,7.185729,-1.315174,-0.212167,13087571,0.000125,11.610430
1606919,3491668,1.234470,0.403945,12.0,10.0,2.294323,10.003275,1675.340897,0.005216,0.015873,2.584891,0.997303,0.061882,6035096,0.000150,7.521060


In [93]:
with h5py.File('Galaxies/LISA_band_FZ_alpha25_Z.hdf', 'r') as f:
    print(f['data']['table'].attrs['values_block_1_kind'].decode('ascii'))
    


(lp0
Vmass_1
p1
aVmass_2
p2
aVkstar_1
p3
aVkstar_2
p4
aVsep
p5
aVmet
p6
aVtphys
p7
aVrad_1
p8
aVrad_2
p9
aVxGx
p10
aVyGx
p11
aVzGx
p12
aVf_gw
p13
aVdist_sun
p14
a.


In [59]:
labels_gb = ['Amplitude', 'EclipticLatitude', 'EclipticLongitude', 'Frequency', 'FrequencyDerivative', 'Inclination', 'InitialPhase', 'Polarization']

In [9]:
def get_amplitude(dat):
    mc = (dat.mass_1.values*dat.mass_2.values)**(3/5) / (dat.mass_1.values + dat.mass_2.values)**(1/5) * u.Msun
    term1 = 64/5 * (const.G * mc)**(10/3)
    term2 = (np.pi*dat.f_gw.values*u.s**(-1))**(4/3)
    denom1 = const.c**8*(dat.dist_sun.values*u.kpc)**2
    amplitude = np.sqrt(term1.to(u.m**10/u.s**(20/3)) * term2 / denom1.to(u.m**10/u.s**8))
    return amplitude

def get_Gx_positions(dat):
    galcen = Galactocentric(x=dat.xGx.values*u.kpc, y=dat.yGx.values*u.kpc, z=dat.zGx.values*u.kpc)
    icrs = galcen.transform_to(ICRS())
    ecl = icrs.transform_to(GeocentricTrueEcliptic())
    return ecl.lon.to(u.rad), ecl.lat.to(u.rad)

def get_chirp(dat):
    mc = (dat.mass_1.values*dat.mass_2.values)**(3/5) / (dat.mass_1.values + dat.mass_2.values)**(1/5) * u.Msun
    fgw = dat.f_gw.values*u.s**(-1)
    term1 = (const.G * mc)**(5/3) / (const.c)**5
    term2 = (np.pi * fgw)**(11/3)
    chirp = 96/(5*np.pi) * term1 * term2
    return chirp.to(u.s**(-2))

def get_inc_phase_pol(dat):
    inc = np.arccos(np.random.uniform(0, 1, len(dat)))
    phase = np.random.uniform(0, np.pi, len(dat))
    pol = np.random.uniform(0, np.pi, len(dat))
    return inc, phase, pol

def create_dat_in(dat):
    h = get_amplitude(dat)
    lon, lat = get_Gx_positions(dat)
    chirp = get_chirp(dat)
    inc, phase, pol = get_inc_phase_pol(dat)
    dat_in = np.vstack([h, lon.value, lat.value, dat.f_gw.values, chirp.value, inc, phase, pol]).T

    return dat_in

In [26]:
dat_in = create_dat_in(dat)

In [28]:
np.shape(dat_in)

(613089, 8)

In [11]:
from GalacticStochastic import global_file_index as gfi

using mkl fft
mkl fft not available trying numpy


In [38]:
from GalacticStochastic import config_helper as ch
config_dict = ch.get_config_dict_from_file("Galaxies/Galaxy1/run_default_parameters.toml") 




In [39]:
params_gb, ns_got = gfi.get_full_galactic_params(config=config_dict)

Component:  dgb 10000
Component:  igb 10000
Component:  vgb 36
Total 20036
Largest frequency derivative 2.898611360478799e-15
